# Imports

In [1]:
import datetime
import yfinance as yf
from src.data.preprocess import extend_market_data
import src.data.weather_script as ws
import pandas as pd
import os
from src.data import helper
from arch import arch_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import xgboost as xgb


# Loading and preprocessing

Unfortunately, we only have rolling futures data, though preferably we'd get, say, only the futures contract expiring in a specific month.

In [2]:
# Define ticker symbols for corn futures
corn_ticker = "ZC=F"   # Corn Futures (rolling)
corn = yf.Ticker(corn_ticker)
corn_data = corn.history(start ="2019-01-01", end="2025-10-31")

corn_data = extend_market_data(corn_data)

features = list(corn_data.columns)
features.remove('expiry')
corn_data = corn_data[features]


In [3]:
# Defining column names for weather features
weather_features = [
       'average_temperature_distribution_weighted_mean',
       'average_temperature_distribution_weighted_variance',
       'average_temperature_distribution_weighted_std',
       'average_temperature_distribution_weighted_skewness',
       'average_temperature_distribution_weighted_kurtosis',
       'average_temperature_distribution_weighted_median',
       'average_temperature_distribution_min_value',
       'average_temperature_distribution_max_value',
       'maximum_temperature_distribution_weighted_mean',
       'maximum_temperature_distribution_weighted_variance',
       'maximum_temperature_distribution_weighted_std',
       'maximum_temperature_distribution_weighted_skewness',
       'maximum_temperature_distribution_weighted_kurtosis',
       'maximum_temperature_distribution_weighted_median',
       'maximum_temperature_distribution_min_value',
       'maximum_temperature_distribution_max_value',
       'minimum_temperature_distribution_weighted_mean',
       'minimum_temperature_distribution_weighted_variance',
       'minimum_temperature_distribution_weighted_std',
       'minimum_temperature_distribution_weighted_skewness',
       'minimum_temperature_distribution_weighted_kurtosis',
       'minimum_temperature_distribution_weighted_median',
       'minimum_temperature_distribution_min_value',
       'minimum_temperature_distribution_max_value',
       'precipitation_distribution_weighted_mean',
       'precipitation_distribution_weighted_variance',
       'precipitation_distribution_weighted_std',
       'precipitation_distribution_weighted_median',
       'precipitation_distribution_min_value',
       'precipitation_distribution_max_value',
       'snow_distribution_weighted_mean',
       'snow_distribution_weighted_variance', 'snow_distribution_weighted_std',
       'snow_distribution_weighted_median', 'snow_distribution_min_value',
       'snow_distribution_max_value']

Obtaining weather data if it exists in the directory. Otherwise, creates the weather data.
cadot is a nested dictionary with the following structure:

dates -- climate tuples (avg temperature, min temperature, max temperature, precipitation, snow) -- amount of corn crop area with this climate tuple on a given date

This distribution has a lot of data, but we make several projections, and our weather_df will be indexed by dates, and will contain the weighted (by crop area) average, std, and other statistics of each of the 5 climate data.

### Requires cropland layer file downloaded from [USDA](https://www.nass.usda.gov/Research_and_Science/Cropland/Release/index.php) + preprocessing

In [4]:
weather_path = "../cached_weather.pkl"
if os.path.exists(weather_path):
    weather_df = pd.read_pickle(weather_path)
else:
    cadot_path = "../cached_cadot"
    if os.path.exists(cadot_path):
        cadot = ws.load_cadot_by_date()
    else:
        m = 200
        cropValue = 1
        BAP = ws.bigArrayParser("../data/2023_30m_cdls.tif", m, cropValue)
        k = 10
        r = 7
        cadot = BAP.get_area_with_climate(k, r, m, ["IA", "IL", "IN", "MO", "SD", "KS", "MN"], "2019-01-01", "2025-10-31")
        ws.save_cadot_by_date(cadot)
    proj = ws.get_projections_multithreaded(cadot)
    weather_df = pd.DataFrame(ws.get_weather_features_multithreaded(proj)).transpose()
    weather_df.to_pickle(weather_path)

weather_df.index = pd.to_datetime(weather_df.index, format='%Y:%m:%d %H:%M:%S')
all_data = corn_data.join(weather_df, how='left')


Since the futures data is rolling, we need to clear the log return on days when the contract switches. We will lose some data, but it's only a few days.

In [5]:
all_data_cleaned = all_data.copy()
all_data_cleaned.loc[(all_data_cleaned['DTE'] == 0).shift(1, fill_value=False), 'Log_Return'] = 0
all_data_cleaned = all_data_cleaned[-360:]

# Model Training

Our goal will be to predict future volatilities, and use it to generate paths instead of assuming constant volatility as in Black-Scholes. We will do this in the following way.

1. We first fit a GARCH(1,1) volatility model.
2. We use a linear model and use the weather data to fit the residuals of the volatility model. (Specifically, we look at the predicted volatility of fitted GARCH(1,1) vs the square of log return

In [6]:
# Fit GARCH(1,1) model
garch_model = arch_model(all_data_cleaned['Log_Return'].dropna(), vol='GARCH', p=1, q=1)
garch_results = garch_model.fit(disp='off')

# Get the predicted variance from the GARCH model
predicted_volatility = garch_results.conditional_volatility**2

# Calculate the GARCH model's residuals
realized_volatility = all_data_cleaned['Log_Return']**2
garch_error = realized_volatility - predicted_volatility

# Defining residual model
X_weather = all_data_cleaned[weather_features].shift(1)
y_error = garch_error[1:]
y_error, X_weather = y_error.align(X_weather, join='inner')
X_weather = X_weather.loc[y_error.index] # Ensure indices match after potential drops
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor())
])

# Fitting residual model
best_error_model = pipeline.fit(X_weather, y_error)

F:\Anaconda\envs\ErdosQuantFinance\Lib\site-packages\arch\univariate\base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001474. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


# GBM paths and call value approximation

With our fitted GARCH(1,1) model + residual model, we do the following

1. We gather volatility paths using the GARCH(1,1) process on the last date in the yfinance data.
2. We use the residual model + historical weather data to predict the effects of weather on future volatilities.
3. We use these volatility paths to generate price paths for futures
4. We use the Longstaff-Schwartz algorithm for pricing an American call option, and estimate the price of a call on the future.

In [7]:
# Variables
today_date = pd.to_datetime('today')
expiration_date = pd.to_datetime('2026-02-20') # Example expiration
business_days = pd.bdate_range(start=today_date, end=expiration_date)
days_to_expiration = len(business_days) # trading days until expiration for call option expiring on 2/20/26 for a March '26 futures contract

t = 106/252 # trading days
K = 440 # strike price
n_sims = 100000
n_steps = days_to_expiration

In [15]:
march_corn_ticker = "ZCH26.CBT"   # Corn Futures (expiring March '26)
march_corn = yf.Ticker(march_corn_ticker)
march_corn_data = march_corn.history(start ="2024-10-01")
march_corn_data = extend_market_data(march_corn_data)
features = list(march_corn_data.columns)
features.remove('expiry')
march_corn_data = march_corn_data[features]
last_volatility = march_corn_data['Log_Return'][1:].std() * (252 ** (1/2))
risk_free_rate = 0.0380 # risk free rate from https://fred.stlouisfed.org/series/DTB3
start_index_position = all_data_cleaned.index.get_indexer([today_date - datetime.timedelta(days=730)], method='nearest')[0]
previous_weather = all_data_cleaned.iloc[start_index_position : start_index_position + days_to_expiration][weather_features]
S0 = march_corn_data['Close'].iloc[-1]

Note that garch_results and best_error_model are models for *daily* volatility, whereas the input initial volatility is the historical yearly volatility.

In [16]:
vol_path, shocks = helper.simulate_hybrid_vol_paths(garch_results,
                                                    best_error_model,
                                                    last_volatility, t, n_sims, n_steps,
                                                    previous_weather)
paths = helper.GBM_paths_with_volatility_paths(S0, risk_free_rate, t, vol_path, shocks)
Ct_model = LinearRegression()


# Comparison to baseline

Our baseline model will be a Black-Scholes model with volatility obtained from the historical (last year) volatility, and we also have actual prices for calls from [barchart](https://www.barchart.com/futures/quotes/ZCH26/options/mar-26?futuresOptionsView=merged).

Currently (as of 11/06/25), the price of the call option for a futures contract for 5000 bushels of corn is $800. Thus, with our pricing model, we likely have vastly overestimated the price, since we don't expect the premium to be so high. On the other hand, just using Black-Scholes with historical volatility overestimates the price, especially considering that Black-Scholes should estimate the fair market value, without considering premiums.

In [17]:
print(f"The estimated price of the call option for a futures contract for 5000 bushels of corn at strike price ${K:.2f} is ${5000 / 100 * helper.LS_algorithm_call(paths, K, Ct_model):.2f}.")

print(f"The estimated price (using Black-Scholes) of the call option for a futures contract for 5000 bushels of corn at strike price ${K:.2f} is ${5000 / 100 * helper.bs_call(S0, K, last_volatility, t, risk_free_rate):.2f}.")

The estimated price of the call option for a futures contract for 5000 bushels of corn at strike price $440.00 is $1319.43.
The estimated price (using Black-Scholes) of the call option for a futures contract for 5000 bushels of corn at strike price $440.00 is $1031.32.


# Conclusion

The current model, as it is, is not performant and overestimates the price of the call. There are a few likely sources of error:
1. Rolling future data do not extrapolate to a single future data. That is, the volatilities for different future contracts behave differently, especially when it is more than 3 months out.
2. The GARCH(1,1) model simply doesn't fit the volatilities well.
3. The way the weather data was used is too crude, and there could be more information to be extracted from it.

# Future Work

1. We may consider other commodities with more data on its historical prices.
2. We may expand our weather model, and obtain more features from the distribution of weather.
3. We may consider a GARCH-x model, where the exogenous data feeds directly into the volatility model, instead of just modeling the residuals.